# SPARQL results in which we don't know which label goes with which IRI + fuzzy string matching = ???


## some preliminary tuple + list review
I looked at:
- [How do I get the last element of a list?](https://stackoverflow.com/questions/930397/how-do-i-get-the-last-element-of-a-list)
- [Add Element to Tuple in Python](https://sparkbyexamples.com/python/add-element-to-tuple-in-python)
- [Accessing Tuple Elements in Python](https://pieriantraining.com/accessing-tuple-elements-in-python/)

In [1]:
# attempt to slide a tuple as a list --
# [start:stop:stride]
test_tuple = ("http://opaquenamespace.org/ns/creator/ParkerJamiesonK", "Clarence L. Smith (architect, 1894-1951)", "Jamieson K. Parker (architect, 1895-1939)")
for value in test_tuple[1::]:
    print(value)
# huzzah!! 🎉

Clarence L. Smith (architect, 1894-1951)
Jamieson K. Parker (architect, 1895-1939)


In [2]:
dump = input("enter path to RDF dump for collection\n>>>")

import rdflib

g = rdflib.Graph().parse(dump)
result = g.query(
    """SELECT DISTINCT ?onsResource ?creatorDisplay
    WHERE {
        ?s ?p ?onsResource .
        OPTIONAL { ?s <http://opaquenamespace.org/ns/cco_creatorDisplay> ?creatorDisplay . }
        FILTER CONTAINS (str(?onsResource), "http://opaquenamespace.org/ns/")
    }
    """
)

In [3]:
all_ons = [
    # I need a start tuple 
    ("throw_this_one_away", )
]
for row in result:
    if str(row.onsResource) == all_ons[-1][0]:
        add_label_tuple = (str(row.creatorDisplay), )
        all_ons[-1] = all_ons[-1] + add_label_tuple
    elif str(row.creatorDisplay) == '':
        new_iri_only_tuple = (str(row.onsResource), )
        all_ons.append(new_iri_only_tuple)
    else:
        new_iri_label_tuple = (str(row.onsResource), str(row.creatorDisplay))
        all_ons.append(new_iri_label_tuple)

In [4]:
# for item in all_ons:
#     print(item)

## fuzzy string matching
**to pick the label that goes with the IRI (if one exists)**  
*NOTE 🚩 that this is only enabled by the fact that ONS IRI slugs are created from labels with space, punctuation, etc. stripped*  

> ***From [datacamp](https://www.datacamp.com/)***  
> https://www.datacamp.com/tutorial/fuzzy-string-python